In [29]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = input("Enter town number: 1, 2, 3, or 10")

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
file_name = latest_file[18:]

index = int(latest_file[27:-8])
new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)
print(latest_file)

graph_list/Town01\1_graph_v0.adjlist


In [30]:
################# Change start, end point #################
start, end = (547, 645), (3871, 1346)
# start, end = end, start


In [31]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "Forward", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    # fp = final_path[0][::-1]
    # fp = [fp]
    # add_path(G, fp, new_file_name)

    add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    cv2.imwrite(f"test/found/test__{start}_{end}.png", new_img)

Start point: (547, 645)
End point: (3871, 1346)
Finding Path...
Not found!!!
[[(547, 645), (547, 650), (547, 654), (547, 658), (547, 662), (547, 666), (547, 670), (547, 674), (547, 678), (547, 682), (547, 686), (547, 690), (547, 694), (547, 698), (547, 702), (547, 706), (547, 710), (547, 714), (547, 718), (547, 722), (547, 726), (547, 730), (547, 734), (547, 738), (547, 742), (547, 746), (547, 750), (547, 754), (547, 758), (547, 762), (547, 766), (547, 770), (547, 774), (547, 778), (547, 782), (547, 786), (547, 790), (547, 794), (547, 798), (547, 802), (547, 806), (547, 810), (547, 814), (547, 818), (547, 822), (547, 826), (547, 830), (547, 834), (547, 838), (547, 842), (547, 846), (547, 850), (547, 854), (547, 858), (547, 862), (547, 866), (547, 870), (547, 874), (547, 878), (547, 882), (547, 886), (547, 890), (547, 894), (547, 898), (547, 902), (547, 906), (547, 910), (547, 914), (547, 918), (547, 922), (547, 926), (547, 930), (547, 934), (547, 938), (547, 942), (547, 946), (547, 950

In [32]:
############### Quick test for other start point with the same goal point
start, end = (547, 645), (3871, 1346)

fp = nx.shortest_path(G,  start,  end)
print(fp)

NodeNotFound: Either source (547, 645) or target (3871, 1346) is not in G